In [22]:
# IMPORTAR LIBRERÍAS

import os
import requests
import pandas as pd
import re
import nltk
from collections import Counter
import pandas as pd
from nltk.corpus import stopwords

In [23]:
# LEER LAS CANCIONES Y BUSCAR LETRAS

MUSIC_DIR = r"C:\Users\9 ----- SIG\Downloads\musicas\Canciones"
SUPPORTED = (".mp3", ".wav", ".flac", ".m4a", ".ogg", ".webm")

rows = []

def clean_name(name):
    # Quitar extensión
    name = os.path.splitext(name)[0]

    # Quitar cosas comunes
    name = re.sub(r"\(.*?\)|\[.*?\]", "", name)
    name = re.sub(r"official.*|lyrics.*|video.*", "", name, flags=re.I)

    return name.strip()

print("🎤 Buscando letras...\n")

files = [f for f in os.listdir(MUSIC_DIR) if f.lower().endswith(SUPPORTED)]

for i, file in enumerate(files, start=1):
    print(f"[{i}/{len(files)}] {file}")

    base = clean_name(file)

    if " - " not in base:
        rows.append({
            "Archivo": file,
            "Artista": None,
            "Título": None,
            "Letra encontrada": "⚠️ Nombre no reconocible"
        })
        continue

    artist, title = base.split(" - ", 1)

    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"

    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200 and r.json().get("lyrics"):
            lyrics = r.json()["lyrics"]
            found = "✅ Sí"
        else:
            lyrics = None
            found = "❌ No"
    except:
        lyrics = None
        found = "❌ Error"

    rows.append({
        "Archivo": file,
        "Artista": artist.strip(),
        "Título": title.strip(),
        "Letra encontrada": found,
        "Longitud letra": len(lyrics) if lyrics else 0
    })

print("\n✅ Búsqueda terminada")

df_lyrics = pd.DataFrame(rows)
df_lyrics

🎤 Buscando letras...

[1/104] Alter Bridge - Addicted To Pain.mp3
[2/104] Alter Bridge - Blackbird.mp3
[3/104] Alter Bridge - Metalingus.mp3
[4/104] Alter Bridge - Rise Today.mp3
[5/104] Assassination Classroom - Question.mp3
[6/104] Avenged Sevenfold - And All Things Will End.mp3
[7/104] Avenged Sevenfold - Chapter Four On Screen.mp3
[8/104] Avenged Sevenfold - Desecrate Through Reverance On Screen.mp3
[9/104] Avenged Sevenfold - I Won't See You Tonight.mp3
[10/104] Avenged Sevenfold - Nightmare.mp3
[11/104] Avenged Sevenfold - Second Heartbeat On Screen.mp3
[12/104] Avenged Sevenfold - Unholy Confessions.mp3
[13/104] Benson Boone - Beautiful Things.mp3
[14/104] Bersuit Vergarabat - Sr Cobranza.m4a
[15/104] Black Veil Brides - Fallen Angels.mp3
[16/104] Black Veil Brides - In The End.m4a
[17/104] Calibre 50 - A La Antigüita.mp3
[18/104] Camila - Solo Para Tí.mp3
[19/104] Cuarteto De Nos - Lo Malo De Ser Bueno.mp3
[20/104] Death Note - What's Up People.mp3
[21/104] Disturbed - Decadenc

,Archivo,Artista,Título,Letra encontrada,Longitud letra
0,Alter Bridge - Addicted To Pain.mp3,Alter Bridge,Addicted To Pain,❌ Error,0.0
1,Alter Bridge - Blackbird.mp3,Alter Bridge,Blackbird,✅ Sí,1082.0
2,Alter Bridge - Metalingus.mp3,Alter Bridge,Metalingus,✅ Sí,1027.0
3,Alter Bridge - Rise Today.mp3,Alter Bridge,Rise Today,❌ Error,0.0
4,Assassination Classroom - Question.mp3,Assassination Classroom,Question,❌ Error,0.0
...,...,...,...,...,...
99,Three Days Grace - Pain.mp3,Three Days Grace,Pain,✅ Sí,1664.0
100,Three Days Grace - Time Of Dying.mp3,Three Days Grace,Time Of Dying,❌ Error,0.0
101,Wisin y Yandel - Nadie Como Tu.m4a,Wisin y Yandel,Nadie Como Tu,❌ Error,0.0
102,WWE - Christian Cancion.mp3,WWE,Christian Cancion,❌ No,0.0


In [25]:
# ANALIZADOR DE TEXTO

nltk.download("stopwords")

stop_words = set(stopwords.words("english"))

lyrics_text = []

[nltk_data] Downloading package stopwords to C:\Users\9 -----
[nltk_data]     SIG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# ANALIZAR LETRAS

for row in df_lyrics.itertuples():
    if row._4 != "✅ Sí":  # Letra encontrada
        continue

    artist = row.Artista
    title = row.Título

    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"

    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            lyrics = r.json().get("lyrics", "")
            lyrics_text.append(lyrics)
    except:
        pass

print(f"🎵 Letras analizadas: {len(lyrics_text)}")

🎵 Letras analizadas: 12


In [27]:
# ANÁLISIS DE PALABRAS

def clean_lyrics(text):
    text = text.lower()
    text = re.sub(r"\[.*?\]", "", text)   # [chorus], etc
    text = re.sub(r"[^a-z\s]", "", text)  # solo letras
    words = text.split()
    words = [w for w in words if w not in stop_words and len(w) > 2]
    return words

all_words = []

for lyric in lyrics_text:
    all_words.extend(clean_lyrics(lyric))

print(f"🧠 Palabras totales analizadas: {len(all_words)}")

🧠 Palabras totales analizadas: 1826


In [28]:
# FRECUENCIA DE PALABRAS

counter = Counter(all_words)

top_words = counter.most_common(30)

df_words = pd.DataFrame(top_words, columns=["Palabra", "Frecuencia"])
df_words

,Palabra,Frecuencia
0,know,24
1,away,23
2,find,23
3,dont,23
4,fui,22
5,want,21
6,head,21
7,falling,21
8,another,20
9,take,20


In [29]:
# AGRUPACIÓN DE TEMAS

themes = {
    "amor_relaciones": ["love", "heart", "feel", "hold", "touch"],
    "dolor_tristeza": ["pain", "hurt", "cry", "alone", "tears", "broken"],
    "ira_conflicto": ["hate", "fight", "rage", "kill", "war"],
    "existencial": ["life", "time", "world", "inside", "mind", "soul"],
    "escape_libertad": ["free", "run", "away", "fly", "leave"]
}

theme_counts = {}

for theme, keywords in themes.items():
    theme_counts[theme] = sum(counter[k] for k in keywords)

df_themes = pd.DataFrame(
    theme_counts.items(),
    columns=["Tema", "Frecuencia"]
).sort_values(by="Frecuencia", ascending=False)

df_themes

,Tema,Frecuencia
3,existencial,49
4,escape_libertad,44
1,dolor_tristeza,34
0,amor_relaciones,12
2,ira_conflicto,7


In [30]:
# RESUMEN DE PERFIL

top_theme = df_themes.iloc[0]["Tema"]

print("🎧 PERFIL DE ESCUCHA")
print("------------------")
print(f"Tema dominante: {top_theme}")
print("Palabras más comunes:")
for w, c in top_words[:10]:
    print(f"- {w}")

🎧 PERFIL DE ESCUCHA
------------------
Tema dominante: existencial
Palabras más comunes:
- know
- away
- find
- dont
- fui
- want
- head
- falling
- another
- take
